VGG16  com Regulariza玢o + Data Augmentation + Dropout 0.5

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(2018)

%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras import regularizers
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten


import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.preprocessing.image import ImageDataGenerator

Fazendo a leitura dos csv's para conseguir os nomes das imagens e suas ra鏰s.

In [ ]:
df_train = pd.read_csv('../input/dog-breed-identification/labels.csv') # arquivo com os nomes das imagens e as suas ra鏰s para o treino
df_test = pd.read_csv('../input/dog-breed-identification/sample_submission.csv') # arquivo com os nomes das imagens de teste

In [ ]:
df_train.head()

In [ ]:
df_test.head()

As ra鏰s precisam estar usando o *one-hot encode*

In [ ]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

A seguinte iremos ler as imagens de treino e teste.

In [ ]:
im_size = 197

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/dog-breed-identification/train/{}.jpg'.format(f))
    x_train.append(cv2.resize(img, (im_size, im_size)))
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1

In [ ]:
# apagando df_train para diminuir uso da mem髍ia
del df_train

In [ ]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/dog-breed-identification/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

In [ ]:
num_class = 120

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train,shuffle=True,  test_size=0.2, random_state=1)

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            rotation_range=30,
                            vertical_flip=False,
                            horizontal_flip=True)

datagen.fit(X_train)

In [ ]:
train_generator = datagen.flow(np.array(X_train), np.array(Y_train), 
                               batch_size=32) 

In [ ]:
def create_my_model(use_regularizer, optimizer):
    base_model = VGG16(weights="../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",include_top=False, input_shape=(im_size, im_size, 3))
    dropout = base_model.output
    dropout = Dropout(0.5)(dropout)
    model_with_dropout = Model(inputs=base_model.input, outputs=dropout)
        
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(num_class, activation='softmax', kernel_regularizer=regularizers.l2(0.0015), activity_regularizer=regularizers.l1(0.0015))(x)
    
    my_model = Model(inputs=base_model.input, outputs=predictions)
    
#     for layer in my_model.layers:
#         layer.treinable = False
    
    my_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return my_model


In [ ]:
def gerar_grafico(historia, titulo):
    plt.plot(historia.history['acc'])
    plt.plot(historia.history['val_acc'])
    plt.title('Acur醕ia ' + titulo)
    plt.ylabel('Acur醕ia')
    plt.xlabel('蓀ocas')
    plt.legend(['treino', 'valida玢o'], loc='upper left')
    plt.show()
    plt.plot(historia.history['loss'])
    plt.plot(historia.history['val_loss'])
    plt.title('Loss ' + titulo)
    plt.ylabel('Loss')
    plt.xlabel('蓀ocas')
    plt.legend(['treino', 'valida玢o'], loc='upper left')
    plt.show()

In [ ]:
model_rmsprop_com_regularizador = create_my_model(use_regularizer=True, optimizer='rmsprop')
model_sgd_com_regularizador = create_my_model(use_regularizer=True, optimizer='sgd')

In [ ]:
history_rmsprop_com_regularizador = model_rmsprop_com_regularizador.fit_generator(
    train_generator,
    epochs=10, steps_per_epoch=len(X_train) / 18, #len(X_train) / 18,
    validation_data=(np.array(X_train), np.array(Y_train)), validation_steps=len(X_valid) / 18)

preds = model_rmsprop_com_regularizador.predict(np.array(x_test), verbose=1)

gerar_grafico(history_rmsprop_com_regularizador, 
              "VGG16 with RMSprop")

sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)

sub.to_csv("predictions_vgg16_with_RMSProp.csv")

model_rmsprop_com_regularizador.save('vgg16_with_RMSProp.h5')


In [ ]:
history_sgd_com_regularizador = model_sgd_com_regularizador.fit_generator(
    train_generator,
    epochs=10, steps_per_epoch=len(X_train) / 18, #len(X_train) / 18,
    validation_data=(np.array(X_train), np.array(Y_train)), validation_steps=len(X_valid) / 18)

preds = model_sgd_com_regularizador.predict(np.array(x_test), verbose=1)

gerar_grafico(history_sgd_com_regularizador, 
              "VGG16 com SGD")

sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)

sub.to_csv("output_sgd_v2_com_data_augmentation_e_sem_regularizador.csv")

model_sgd_com_regularizador.save('sgd_v2_com_data_augmentation_e_sem_regularizador.h5')